In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


base_url="https://pokemondb.net/pokedex/all"
req_all=requests.get(base_url)
cont_all=req_all.content

soup_all=BeautifulSoup(cont_all,"html.parser")
all=soup_all.find_all("table",{"id":"pokedex"})[0].find_all("tr")

pokedex=[]

for item in all:    
    dict={}
    poke_names=item.find_all("a",{"class":"ent-name"})
    if poke_names:
        dict["Pokemon"]=poke_names[0].text
        
    poke_id=item.find_all("span",{"class":"infocard-cell-data"})
    if poke_id:
        dict["Ndex"]=poke_id[0].text
    poke_type=item.find_all("a",{"class":"type-icon"})
    if poke_type:
        init_itr=True
        for i in poke_type:
            if init_itr:
                  typing=i.text
            else :
                typing=typing+"/"+i.text
            init_itr=False   
        dict["Type"]=typing
    poke_total=item.find_all("td",{"class":"cell-total"})
    if poke_total:
        dict["Total"]=poke_total[0].text
        
    poke_stats=item.find_all("td",{"class":"cell-num"})
    if poke_stats:
        dict["Hp"]=poke_stats[1].text
        dict["Attack"]=poke_stats[2].text
        dict["Defense"]=poke_stats[3].text
        dict["Special Attack"]=poke_stats[4].text    
        dict["Special Defense"]=poke_stats[5].text
        dict["Speed"]=poke_stats[6].text
    
    pokedex.append(dict)
    



df=pd.DataFrame(pokedex)
df=df[df["Ndex"].notna()]

df=df.set_index('Ndex')
df1=df.drop_duplicates()
df1.to_csv("Pokemon.csv")
df1.to_json("Pokemon.json",orient="records")